In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex Prediction with Custom TorchServe Container

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/community-content/pytorch_image_classification_single_gpu_with_vertex_sdk_and_torchserve/vertex_prediction_with_custom_torchserve_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Setup

In [ ]:
PROJECT_ID = "YOUR PROJECT ID"
BUCKET_NAME = "gs://YOUR BUCKET NAME"
REGION = "YOUR REGION"
SERVICE_ACCOUNT = "YOUR SERVICE ACCOUNT"

In [ ]:
content_name = "pt-img-cls-gpu-cust-cont-torchserve"

### Training Artifact

In [ ]:
gcs_output_uri_prefix = f"{BUCKET_NAME}/{content_name}"

In [ ]:
! gsutil ls $gcs_output_uri_prefix

## Vertex Prediction using Custom TorchServe Container

### Test Sample Image

In [ ]:
! curl -O https://raw.githubusercontent.com/alvarobartt/pytorch-model-serving/master/images/sample.jpg

In [ ]:
! ls sample.jpg

In [ ]:
%run convert_b64.py

In [ ]:
! ls sample_b64.json

### Model Archive for TorchServe

In [ ]:
! gsutil cp -r $gcs_output_uri_prefix/model ./model_server/

In [ ]:
! ls ./model_server/model/

In [ ]:
! cd model_server && torch-model-archiver \
     --model-name antandbee \
     --version 1.0 \
     --serialized-file ./model/antandbee.pth \
     --model-file ./model.py \
     --handler ./handler.py \
     --extra-files ./index_to_name.json \
     -f

In [ ]:
! ls model_server/antandbee.mar

### Option: TorchServe Local Run

```
cd model_server
torchserve --model-store ./ \
  --ts-config ./config.properties \
  --models antandbee=antandbee.mar

curl http://localhost:8080/ping

curl http://127.0.0.1:8081/models/antandbee

curl -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @sample_b64.json \
  http://localhost:8080/predictions/antandbee

torchserve --stop


! rm model_server/antandbee.mar
! rm -rf model_server/logs

```

### Custom TorchServe Container

In [ ]:
hostname = "gcr.io"
tag = "latest"

model_name = "antandbee"
image_name_serve = content_name + "-" + model_name
custom_container_image_uri_serve=f"{hostname}/{PROJECT_ID}/{image_name_serve}:{tag}"

In [ ]:
! cd model_server && docker build -t $custom_container_image_uri_serve -f Dockerfile .

In [ ]:
! rm -rf ./model_server/model/

In [ ]:
! docker run \
    --rm -it \
    -d \
    --name ts_antandbee \
    -p 8080:8080 \
    -p 8081:8081 \
    $custom_container_image_uri_serve

In [ ]:
! curl http://localhost:8080/ping

In [ ]:
! curl http://127.0.0.1:8081/models/antandbee

In [ ]:
! curl -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @sample_b64.json \
  localhost:8080/predictions/antandbee

In [ ]:
! docker stop ts_antandbee

In [ ]:
! docker push $custom_container_image_uri_serve

In [ ]:
! gcloud container images list --repository $hostname/$PROJECT_ID

### Initialize Vertex SDK

In [ ]:
! pip install -r requirements.txt

In [ ]:
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_NAME,
    location=REGION,
)

### Create a Vertex Model with Custom TorchServe Container

In [ ]:
model_display_name = image_name_serve

In [ ]:
model = aiplatform.Model.upload(
    display_name=model_display_name,
    serving_container_image_uri=custom_container_image_uri_serve,
    serving_container_ports=[8080],
    serving_container_predict_route=f"/predictions/{model_name}",
    serving_container_health_route="/ping",
)

### Create a Vertex Endpoint for Online Prediction

In [ ]:
endpoint = model.deploy(
    machine_type="n1-standard-4",
)

In [ ]:
endpoint.resource_name

In [ ]:
import base64

def convert_b64(input_file_name):
  """Open image and convert it to Base64"""
  with open(input_file_name, 'rb') as input_file:
    jpeg_bytes = base64.b64encode(input_file.read()).decode('utf-8')
  return jpeg_bytes

In [ ]:
image_file_name = "./sample.jpg"
instance = {"data": {"b64": convert_b64(image_file_name)}}
prediction = endpoint.predict(instances=[instance])
print(prediction)

## Clean Up

In [ ]:
! gsutil rm -rf $gcs_output_uri_prefix

In [ ]:
! rm sample.jpg
! rm sample_b64.json

In [ ]:
! rm model_server/antandbee.mar
